In [1]:
using CSV
using DataFrames
using GLM
using Optim
using Statistics
using ForwardDiff

# The Nerlove Model

### Theoretical Background
For a firm that takes input prices $w$ and the output level $q$
as given, the cost minimization problem is to choose the quantities
of inputs $x$ to solve the problem
$$
\min_{x}w'x
$$
 subject to the restriction
$$
f(x)=q.
$$
 The solution is the vector of factor demands $x(w,q)$. The cost
function is obtained by substituting the factor demands into the
criterion function: 
$$
Cw,q)=w'x(w,q).
$$
 
- **Monotonicity** Increasing factor prices cannot decrease cost, so 
$$\frac{\partial C(w,q)}{\partial w}\geq0$$
Remember that these derivatives give the conditional factor demands
(Shephard's Lemma).
- **Homogeneity** The cost function is homogeneous of degree 1 in input prices: $C(tw,q)=tC(w,q)$ where $t$ is a scalar constant. This is because the factor demands are homogeneous of degree zero in factor prices - they only depend upon relative prices.
- **Returns to scale** The returns to scale parameter $\gamma$ is defined as the inverse of the elasticity of cost with respect to output:
$$
\gamma=\left(\frac{\partial C(w,q)}{\partial q}\frac{q}{C(w,q)}\right)^{-1}
$$
Constant returns to scale is the case where increasing production
$q$ implies that cost increases in the proportion 1:1. If this is
the case, then $\gamma=1$.

#### Cobb-Douglas functional form

The Cobb-Douglas functional form is linear in the logarithms of the
regressors and the dependent variable. For a cost function, if there
are $g$ factors, the Cobb-Douglas cost function has the form

$$
C=Aw_{1}^{\beta_{1}}...w_{g}^{\beta_{g}}q^{\beta_{q}}e^{\varepsilon}
$$
What is the elasticity of $C$ with respect to $w_{j}$?
\begin{eqnarray*}
e_{w_{j}}^{C} & = & \left(\frac{\partial C}{\partial_{W_{J}}}\right)\left(\frac{w_{j}}{C}\right)\\
 & = & \beta_{j}Aw_{1}^{\beta_{1}}.w_{j}^{\beta_{j}-1}..w_{g}^{\beta_{g}}q^{\beta_{q}}e^{\varepsilon}\frac{w_{j}}{Aw_{1}^{\beta_{1}}...w_{g}^{\beta_{g}}q^{\beta_{q}}e^{\varepsilon}}\\
 & = & \beta_{j}
\end{eqnarray*}
This is one of the reasons the Cobb-Douglas form is popular - the
coefficients are easy to interpret, since they are the elasticities
of the dependent variable with respect to the explanatory variable.
Not that in this case,
\begin{eqnarray*}
e_{w_{j}}^{C} & = & \left(\frac{\partial C}{\partial_{W_{J}}}\right)\left(\frac{w_{j}}{C}\right)\\
 & = & x_{j}(w,q)\frac{w_{j}}{C}\\
 & \equiv & s_{j}(w,q)
\end{eqnarray*}
the cost share of the $j^{th}$ input. So with a Cobb-Douglas
cost function, $\beta_{j}=s_{j}(w,q)$. The cost shares are constants.

Note that after a logarithmic transformation we obtain
$$
\ln C=\alpha+\beta_{1}\ln w_{1}+...+\beta_{g}\ln w_{g}+\beta_{q}\ln q+\epsilon
$$
where $\alpha=\ln A$ . So we see that the transformed model is linear
in the logs of the data.

One can verify that the property of HOD1 implies that 
$$
\sum_{i=1}^{g}\beta_{i}=1
$$
In other words, the cost shares add up to 1. 

The hypothesis that the technology exhibits CRTS implies that 
$$
\gamma=\frac{1}{\beta_{q}}=1
$$
so $\beta_{q}=1.$ Likewise, monotonicity implies that the coefficients
$\beta_{i}\geq0,i=1,...,g$.


### The Nerlove Data
The file contains data on 145 electric utility companies' cost of production,
output and input prices. The data are for the U.S., and were collected
by M. Nerlove. The observations are by row, and the columns are 
- COMPANYCOST $(C)$
- OUTPUT $(Q)$ 
- PRICE OF LABOR $(P_{L})$
- PRICE OF FUEL $(P_{F})$
- PRICE OF CAPITAL$(P_{K})$ 

Note that the data are sorted by output level (the third column).

We will estimate the Cobb-Douglas model 
$$\ln C=\beta_{1}+\beta_{Q}\ln Q+\beta_{L}\ln P_{L}+\beta_{F}\ln P_{F}+\beta_{K}\ln P_{K}+\epsilon\label{simple nerlove model}
$$ by OLS.


## OLS

In [2]:
data = DataFrame(CSV.File("../data/nerlove.csv"))
first(data,6)

,firm,cost,output,labor,fuel,capital
,Int64,Float64,Int64,Float64,Float64,Int64
1,101,0.082,2,2.09,17.9,183
2,102,0.661,3,2.05,35.1,174
3,103,0.99,4,2.05,35.1,171
4,104,0.315,4,1.83,32.2,166
5,105,0.197,5,2.12,28.6,233
6,106,0.098,9,2.12,28.6,195


In [3]:
data = log.(data[:,[:cost,:output,:labor,:fuel,:capital]])
first(data,6)

,cost,output,labor,fuel,capital
,Float64,Float64,Float64,Float64,Float64
1,-2.50104,0.693147,0.737164,2.8848,5.20949
2,-0.414001,1.09861,0.71784,3.5582,5.15906
3,-0.0100503,1.38629,0.71784,3.5582,5.14166
4,-1.15518,1.38629,0.604316,3.47197,5.11199
5,-1.62455,1.60944,0.751416,3.35341,5.45104
6,-2.32279,2.19722,0.751416,3.35341,5.273


In [4]:
ols = lm(@formula(cost~output+labor+fuel+capital),data)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.CholeskyPivoted{Float64,Array{Float64,2}}}},Array{Float64,2}}

cost ~ 1 + output + labor + fuel + capital

Coefficients:
──────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error      t  Pr(>|t|)  Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────
(Intercept)  -3.5265     1.77437    -1.99    0.0488  -7.03452   -0.0184845
output        0.720394   0.0174664  41.24    <1e-79   0.685862   0.754926
labor         0.436341   0.291048    1.50    0.1361  -0.139076   1.01176
fuel          0.426517   0.100369    4.25    <1e-4    0.228082   0.624952
capital      -0.219888   0.339429   -0.65    0.5182  -0.890957   0.45118
──────────────────────────────────────────────────────────────────────────

In [15]:
n = size(data,1)
y = data[:,1]
x = data[:,2:end]
x[!,:intercept]=ones(size(data,1))
x = x[!,[:intercept,:output,:labor,:fuel,:capital]]

y = convert(Array,y)
x = convert(Array,x)
inv(x'*x)*x'*y

5-element Array{Float64,1}:
 -3.5265028449802216
  0.7203940758797012
  0.4363412007892406
  0.4265169530627446
 -0.2198883507567723

## MLE

In [6]:
function fminunc(obj, x; tol = 1e-08)
    results = Optim.optimize(obj, x, LBFGS(), 
                            Optim.Options(
                            g_tol = tol,
                            x_tol=tol,
                            f_tol=tol))
    return results.minimizer, results.minimum, Optim.converged(results)
    #xopt, objvalue, flag = fmincon(obj, x, tol=tol)
    #return xopt, objvalue, flag
end

fminunc (generic function with 1 method)

In [16]:
function normal(theta, y, x)
    b = theta[1:end-1]
    s = theta[end][1]
    e = (y - x*b)./s
    logdensity = -log.(sqrt.(2.0*pi)) .- 0.5*log(s.^2) .- 0.5*e.*e
end

normal (generic function with 1 method)

In [8]:
function mle(model, θ)
    avg_obj = θ -> -mean(vec(model(θ))) # average log likelihood
    thetahat, objvalue, converged = fminunc(avg_obj, θ) # do the minimization of -logL
    objvalue = -objvalue
    obj = θ -> vec(model(θ)) # unaveraged log likelihood
    n = size(obj(θ),1) # how many observations?
    scorecontrib = ForwardDiff.jacobian(obj, vec(thetahat))
    I = cov(scorecontrib)
    J = ForwardDiff.hessian(avg_obj, vec(thetahat))
    Jinv = inv(J)
    V= Jinv*I*Jinv/n
    return thetahat, objvalue, V, converged
end

mle (generic function with 1 method)

In [19]:
theta = [zeros(size(x,2)); 1.0] # start values for estimation
thetahat, objvalue, V, converged = mle(normal, theta)

LoadError: [91mMethodError: objects of type Array{Float64,1} are not callable[39m
[91mUse square brackets [] for indexing an Array.[39m

In [10]:
thetahat

LoadError: [91mUndefVarError: thetahat not defined[39m

In [11]:
converged

LoadError: [91mUndefVarError: converged not defined[39m